***Participantes (RM - NOME):***<br>

346269 - KAIQUE VALIM<br>
346320 - VITOR VENARDOS<br>
346364 - ATONIO JUNIOR<br>
346851 - GUSTAVO SECCO<br>
346922 - RENAN COVRRE<br>

# **Criar um classificador de chamados aplicando técnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do modelo seu modelo.​

O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

# Bibliotecas

In [ ]:
# Imports do sistema operacional (OS)
import os

# Imports do pandas
import pandas as pd

# Imports relacionados a expressões regulares
import re

# Imports relacionados ao processamento de linguagem natural (NLP)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Imports relacionados ao machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


# Import

In [20]:
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')
df

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."
...,...,...,...,...
21067,3094545,2018-12-07T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Depois de ser um cliente de cartão de persegui...
21068,3091984,2018-12-05T12:00:00-05:00,Roubo / Relatório de disputa,"Na quarta -feira, xx/xx/xxxx, liguei para o Ch..."
21069,3133355,2019-01-25T12:00:00-05:00,Roubo / Relatório de disputa,Não estou familiarizado com o XXXX Pay e não e...
21070,3110963,2018-12-27T12:00:00-05:00,Outros,Eu tive crédito impecável por 30 anos. Eu tive...


# EDA

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


In [12]:
# Verificar a distribuição das categorias
category_counts = df['categoria'].value_counts()

# Verificar valores ausentes
missing_values = df.isnull().sum()

# Calcular a contagem média de palavras na descrição
df['word_count'] = df['descricao_reclamacao'].apply(lambda x: len(str(x).split()))
average_word_count = df['word_count'].mean()

category_counts, missing_values, average_word_count


(Serviços de conta bancária             5161
 Cartão de crédito / Cartão pré-pago    5006
 Roubo / Relatório de disputa           4822
 Hipotecas / Empréstimos                3850
 Outros                                 2233
 Name: categoria, dtype: int64,
 id_reclamacao           0
 data_abertura           0
 categoria               0
 descricao_reclamacao    0
 dtype: int64,
 247.7824126803341)

# Stopwords e Lemmatizer

In [13]:
# Baixar os dados necessários do NLTK
nltk.download(['stopwords', 'wordnet'])

# Inicializa o lematizador
wnl = WordNetLemmatizer()

# Define uma função para limpar o texto
def clean_text(text):
    # Remove pontuações e caracteres especiais (mantém apenas letras do alfabeto)
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    # Converte para minúsculas
    text = text.lower()
    
    # Tokeniza e remove as palavras de parada
    text = text.split()
    text = [wnl.lemmatize(word) for word in text if not word in set(stopwords.words('portuguese'))]
    
    # Une as palavras de volta em uma única string
    text = ' '.join(text)
    
    return text

# Limpa a coluna descrição
df['descricao_reclamacao'] = df['descricao_reclamacao'].apply(clean_text)

df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KAIQUEHENRIQUEVALIM\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KAIQUEHENRIQUEVALIM\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package wordnet is already up-to-date!


,id_reclamacao,data_abertura,categoria,descricao_reclamacao,word_count
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,bom dia nome xxxx xxxx agrade voc puder ajudar...,88
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,atualizei cart xxxx xxxx xx xx informado agent...,58
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,cart chase relatado xx xx entanto pedido fraud...,33
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,xx xx enquanto tentava reservar ticket xxxx xx...,276
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,neto d cheque depositei conta chase fundo limp...,108


# LabelEncoder

In [14]:
# Inicializa o codificador de rótulos
le = LabelEncoder()

# Codifica os rótulos (categorias)
df['categoria'] = le.fit_transform(df['categoria'])

# Divide os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df['descricao_reclamacao'], df['categoria'], test_size=0.25, random_state=42)

# Verifica as dimensões dos conjuntos resultantes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((15804,), (5268,), (15804,), (5268,))

# TF-IDF

In [15]:
# Inicializa o vetorizador TF-IDF
vectorizer = TfidfVectorizer()

# Ajusta o vetorizador aos dados de treino e transforma os dados de treino
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transforma os dados de teste
X_test_tfidf = vectorizer.transform(X_test)

# Verifica as dimensões das matrizes TF-IDF resultantes
X_train_tfidf.shape, X_test_tfidf.shape


((15804, 32463), (5268, 32463))

# SMOTE

In [16]:
# Inicializa SMOTE
smote = SMOTE()

# Adapta SMOTE aos dados de treino
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, y_train)

# GridSearch

In [18]:
# Define os modelos
log_reg = LogisticRegression()
rand_forest = RandomForestClassifier()

# Define os parâmetros a serem buscados
log_reg_params = { 'C': [0.1, 1, 10] }
rand_forest_params = { 'n_estimators': [100, 200, 300] }

# Inicializa GridSearchCV para regressão logística
grid_search_log_reg = GridSearchCV(estimator=log_reg, param_grid=log_reg_params, scoring='f1_weighted', cv=5)

# Ajusta GridSearchCV aos dados de treinamento balanceados
grid_search_log_reg.fit(X_train_balanced, y_train_balanced)

# Imprime os melhores parâmetros e a melhor pontuação para regressão logística
print(grid_search_log_reg.best_params_)
print(grid_search_log_reg.best_score_)

# Inicializa GridSearchCV para floresta aleatória
grid_search_rand_forest = GridSearchCV(estimator=rand_forest, param_grid=rand_forest_params, scoring='f1_weighted', cv=5)

# Ajusta GridSearchCV aos dados de treinamento balanceados
grid_search_rand_forest.fit(X_train_balanced, y_train_balanced)

# Imprime os melhores parâmetros e a melhor pontuação para floresta aleatória
print(grid_search_rand_forest.best_params_)
print(grid_search_rand_forest.best_score_)


c:\Users\KAIQUEHENRIQUEVALIM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\KAIQUEHENRIQUEVALIM\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

{'C': 10}
0.9284321214638609
{'n_estimators': 300}
0.8885586516849966


# Resultados

In [19]:
# Use o melhor modelo de regressão logística para fazer previsões no conjunto de teste
best_log_reg = grid_search_log_reg.best_estimator_
y_pred_log_reg = best_log_reg.predict(X_test_tfidf)

# Gere e imprima o relatório de classificação para a regressão logística
print(classification_report(y_test, y_pred_log_reg, target_names=le.classes_))

# Use o melhor modelo de floresta aleatória para fazer previsões no conjunto de teste
best_rand_forest = grid_search_rand_forest.best_estimator_
y_pred_rand_forest = best_rand_forest.predict(X_test_tfidf)

# Gere e imprima o relatório de classificação para a floresta aleatória
print(classification_report(y_test, y_pred_rand_forest, target_names=le.classes_))


                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.92      0.91      0.92      1290
            Hipotecas / Empréstimos       0.92      0.92      0.92       922
                             Outros       0.87      0.90      0.89       549
       Roubo / Relatório de disputa       0.89      0.88      0.89      1204
         Serviços de conta bancária       0.92      0.92      0.92      1303

                           accuracy                           0.91      5268
                          macro avg       0.90      0.91      0.91      5268
                       weighted avg       0.91      0.91      0.91      5268

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.85      0.89      0.87      1290
            Hipotecas / Empréstimos       0.89      0.89      0.89       922
                             Outros       0.83      0.65      0.73      

Melhor modelo: LogisticRegression(C=10)

F1 weighted médio = 0.91